## Mean of ERC20 transfer amounts

This notebook shows how to calculate the mean of ERC20 transfer amounts, pulling data in from a Postgres database. First we install and get the necessary libraries running. 
The first of which is [shovel](https://indexsupply.com/shovel/docs/#getting-started), which is a library that allows us to pull data from the Ethereum blockchain into a Postgres database.

Make sure you install postgres if needed https://indexsupply.com/shovel/docs/#getting-started. 



In [ ]:
import os
import getpass
import json


# swap out for the relevant linux/amd64, darwin/arm64, darwin/amd64, windows/amd64
os.system("curl -LO https://indexsupply.net/bin/1.0/linux/amd64/shovel")
os.system("chmod +x shovel")


os.environ["PG_URL"] = "postgresql://" + getpass.getuser() + ":@localhost:5432/shovel"

# create a config.json file with the following contents
config = {
  "pg_url": "$PG_URL",
  "eth_sources": [
    {"name": "mainnet", "chain_id": 1, "url": "https://ethereum-rpc.publicnode.com"},
    {"name": "base", "chain_id": 8453, "url": "https://base-rpc.publicnode.com"}
  ],
  "integrations": [{
    "name": "usdc_transfer",
    "enabled": True,
    "sources": [{"name": "mainnet"}, {"name": "base"}],
    "table": {
      "name": "usdc",
      "columns": [
        {"name": "log_addr",  "type": "bytea"},
        {"name": "block_num", "type": "numeric"},
        {"name": "f",         "type": "bytea"},
        {"name": "t",         "type": "bytea"},
        {"name": "v",         "type": "numeric"}
      ]
    },
    "block": [
      {"name": "block_num", "column": "block_num"},
      {
        "name": "log_addr",
        "column": "log_addr",
        "filter_op": "contains",
        "filter_arg": [
          "a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
          "833589fCD6eDb6E08f4c7C32D4f71b54bdA02913"
        ]
      }
    ],
    "event": {
      "name": "Transfer",
      "type": "event",
      "anonymous": False,
      "inputs": [
        {"indexed": True,  "name": "from",  "type": "address", "column": "f"},
        {"indexed": True,  "name": "to",    "type": "address", "column": "t"},
        {"indexed": False, "name": "value", "type": "uint256", "column": "v"}
      ]
    }
  }]
}

# write the config to a file
with open("config.json", "w") as f:
  f.write(json.dumps(config))


# print the two env variables
os.system("echo $PG_URL")

os.system("createdb -h localhost -p 5432 shovel")
# equivalent of nohup ./e2pg -reset -e $RLPS_URL -pg $PG_URL &
command = ["./shovel", "-config", "config.json"]
os.spawnl(os.P_NOWAIT, *command)



**Import Dependencies**

In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import ezkl
import torch
import datetime
import pandas as pd
import requests
import json
import os

import logging
# # uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.DEBUG)

print("ezkl version: ", ezkl.__version__)

**Create Computational Graph**

In [ ]:
from torch import nn
import torch


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

    # x is a time series 
    def forward(self, x):
        return [torch.mean(x)]




circuit = Model()




x = 0.1*torch.rand(1,*[1,5], requires_grad=True)

# # print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

circuit.to(device)

# Flips the neural net into inference mode
circuit.eval()

# Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "lol.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

# export(circuit, input_shape=[1, 20])



**Set Data Source and Get Data**

In [ ]:
import getpass
# make an input.json file from the df above
input_filename = os.path.join('input.json')

pg_input_file = dict(input_data = {
  "host": "localhost",
  # make sure you replace this with your own username
  "user": getpass.getuser(),
  "dbname": "shovel",
  "password": "",
  "query": "SELECT v FROM usdc ORDER BY block_num DESC LIMIT 5",
  "port": "5432",
})

json_formatted_str = json.dumps(pg_input_file, indent=2)
print(json_formatted_str)


    # Serialize data into file:
json.dump( pg_input_file, open(input_filename, 'w' ))


In [ ]:
# this corresponds to 4 batches
calibration_filename = os.path.join('calibration.json')

pg_cal_file = dict(input_data = {
  "host": "localhost",
  # make sure you replace this with your own username
  "user": getpass.getuser(),
  "dbname": "shovel",
  "password": "",
  "query": "SELECT v FROM usdc ORDER BY block_num DESC LIMIT 20",
  "port": "5432",
})

    # Serialize data into file:
json.dump( pg_cal_file, open(calibration_filename, 'w' ))

**EZKL Workflow**

In [ ]:
import subprocess
import os

onnx_filename = os.path.join('lol.onnx')
compiled_filename = os.path.join('lol.compiled')
settings_filename = os.path.join('settings.json')

# Generate settings using ezkl
res = ezkl.gen_settings(onnx_filename, settings_filename)

assert res == True

res = ezkl.calibrate_settings(input_filename, onnx_filename, settings_filename, "resources")

assert res == True

In [ ]:

ezkl.compile_circuit(onnx_filename, compiled_filename, settings_filename)

In [ ]:
# generate settings


# show the settings.json
with open("settings.json") as f:
  data = json.load(f)
  json_formatted_str = json.dumps(data, indent=2)

  print(json_formatted_str)

assert os.path.exists("settings.json")
assert os.path.exists("input.json")
assert os.path.exists("lol.onnx")

In [ ]:
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')


# setup the proof
res = ezkl.setup(
        compiled_filename,
        vk_path,
        pk_path
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_filename)



In [ ]:
witness_path = "witness.json"

# generate the witness
res = ezkl.gen_witness(
        input_filename,
        compiled_filename,
        witness_path
    )


In [ ]:
# prove the zk circuit
# GENERATE A PROOF
proof_path = os.path.join('test.pf')


proof = ezkl.prove(
        witness_path,
        compiled_filename,
        pk_path,
        proof_path,
        "single"
    )


print("proved")

assert os.path.isfile(proof_path)



In [ ]:
# kill all shovel process 
os.system("pkill -f shovel")